# Import Libraries

In [ ]:
import os
import numpy as np
import pydicom

import numpy as np
import matplotlib.pyplot as plt
from skimage import img_as_float, exposure
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage.metrics import peak_signal_noise_ratio
from skimage.util import random_noise
import os


import pydicom
import skimage
print(skimage.__version__)

import cv2

# Add Poisson Noise

In [ ]:

def add_poisson_noise(image, noise_level=0.80):
    """
    Add Poisson noise to a 2D image.
    
    Parameters:
    - image: numpy array, the original image.
    - noise_level: fraction of the estimated photon count to use for noise.
    
    Returns:
    - noisy_image: numpy array, the noisy image.
    """
    image = image.astype(np.float32)
    min_val, max_val = np.min(image), np.max(image)

    estimated_photon_count = (image - min_val) / (max_val - min_val) * 1000 
    
    noisy_photon_count = np.random.poisson(estimated_photon_count * noise_level)
    
    noisy_image = noisy_photon_count / 1000 * (max_val - min_val) + min_val
    noisy_image = np.clip(noisy_image, min_val, max_val) 

    return noisy_image.astype(image.dtype)

# Image Directory

In [ ]:
def process_ct_images(input_dir, output_dir, noise_level=0.80):
    """
    Process all DICOM files in the input directory and its subdirectories, 
    add Poisson noise, and save the noisy images to the output directory.

    Parameters:
    - input_dir: path to the directory with original CT-Scan images (can include subdirectories).
    - output_dir: path to save noisy images.
    - noise_level: fraction of the estimated photon count to use for noise.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for root, _, files in os.walk(input_dir):
        for filename in files:
            file_path = os.path.join(root, filename)
            if os.path.isfile(file_path):
                try:
                    dicom = pydicom.dcmread(file_path)
                    image = dicom.pixel_array
                    
                    noisy_image = add_poisson_noise(image, noise_level)
                    
                    relative_path = os.path.relpath(root, input_dir)
                    sub_output_dir = os.path.join(output_dir, relative_path)
                    if not os.path.exists(sub_output_dir):
                        os.makedirs(sub_output_dir)
                    output_path = os.path.join(sub_output_dir, filename + ".dcm" if not filename.endswith(".dcm") else filename)
                    
                    noisy_dicom = dicom
                    noisy_dicom.PixelData = noisy_image.astype(np.uint16).tobytes()
                    noisy_dicom.save_as(output_path)
                    print(f"Processed: {file_path} -> {output_path}")
                except Exception as e:
                    print(f"Skipping {file_path}: Not a valid DICOM file ({e})")




 # Execute

In [ ]:
# Directories
input_directory = "C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (17 feb)/Data/Original/CATHLAB"
output_directory = "C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (17 feb)/Data/Added Noise/CATHLAB/Poisson/CATHLAB Noisy (Poisson 0.80)"

# Add Poisson noise
process_ct_images(input_directory, output_directory, noise_level=0.80)